# Tuning Analysis: Visualizing Hyperparameter Search

This notebook demonstrates how to inspect the results of a parameter tuning session.
Since we don't have a live database of tuning runs yet, we generate **synthetic tuning data** to showcase:
1.  **Parallel Coordinates**: How parameters interact to impact performance.
2.  **Parameter Importance**: Using Random Forest to quantify which parameters matter most.
3.  **Optimization Trace**: How the tuner improved over time.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.inspection import PartialDependenceDisplay

## 1. Generate Synthetic Tuning Data
We simulate a tuning session for NSGA-II on a TSP problem.
Parameters:
- `pop_size`: [50, 200]
- `crossover_prob`: [0.5, 1.0]
- `mutation_prob`: [0.0, 0.2]
- `selection_pressure`: [2, 10]

Metric: **Hypervolume** (maximize). We'll assume a "ground truth" landscape where:
- High `pop_size` is good.
- `crossover_prob` ~ 0.9 is good.
- `mutation_prob` ~ 0.1 is good.

In [ ]:
np.random.seed(42)
n_samples = 200

# Random Search Sampling
data = {
    "pop_size": np.random.randint(50, 201, n_samples),
    "crossover_prob": np.random.uniform(0.5, 1.0, n_samples),
    "mutation_prob": np.random.uniform(0.0, 0.2, n_samples),
    "selection_pressure": np.random.randint(2, 11, n_samples),
}
df = pd.DataFrame(data)

# Ground Truth Function (Synthetic Efficiency)
# HV = Base + contributions - penalties
def ground_truth(row):
    hv = 0.5
    # larger pop is better but diminishing returns
    hv += 0.2 * (row["pop_size"] / 200)
    # Optimal crossover ~ 0.9
    hv -= 2.0 * (row["crossover_prob"] - 0.9)**2
    # Optimal mutation ~ 0.1
    hv -= 5.0 * (row["mutation_prob"] - 0.1)**2
    # Selection pressure doesn't matter much (noise)
    hv += 0.01 * np.random.randn()
    
    # Add some random noise
    hv += 0.02 * np.random.randn()
    return hv

df["hypervolume"] = df.apply(ground_truth, axis=1)
df["iteration"] = np.arange(n_samples)

# Sort by HV to see best
df_sorted = df.sort_values("hypervolume", ascending=False)
print("Top 5 Configurations:")
print(df_sorted.head())

## 2. Optimization Trace
Did we find better solutions over time? (In Random Search, this is just luck, but in active learning it would trend up).

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df["iteration"], df["hypervolume"], 'o', alpha=0.5, label="Evaluations")
plt.plot(df["iteration"], df["hypervolume"].cummax(), 'r-', linewidth=2, label="Best Found")
plt.xlabel("Iteration")
plt.ylabel("Hypervolume")
plt.title("Optimization Trace")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Parallel Coordinates Plot
Visualize high-dimensional data. Best solutions are colored brightly.

In [ ]:
# Normalize inputs for Parallel Coordinates
cols = ["pop_size", "crossover_prob", "mutation_prob", "selection_pressure", "hypervolume"]
df_norm = df[cols].copy()
scaler = MinMaxScaler()
df_norm[cols] = scaler.fit_transform(df_norm[cols])

plt.figure(figsize=(12, 6))
pd.plotting.parallel_coordinates(df_norm, "hypervolume", cols=cols, color=plt.cm.viridis(df_norm["hypervolume"]))
# Note: standard parallel_coordinates uses a categorical class column, which is messy for continuous HV.
# Let's use a custom approach or just plot lines colored by value.
plt.close()

# Better Line Plot approach
plt.figure(figsize=(12, 6))
for i, row in df_norm.iterrows():
    alpha = max(0.1, row["hypervolume"]**2) # emphasize good ones
    color = plt.cm.viridis(row["hypervolume"])
    plt.plot(range(len(cols)-1), row[:-1], color=color, alpha=alpha)

plt.xticks(range(len(cols)-1), cols[:-1])
plt.ylabel("Normalized Value")
plt.title("Parallel Coordinates (Bighter = Better HV)")
plt.colorbar(plt.cm.ScalarMappable(cmap="viridis"), label="Normalized Hypervolume")
plt.show()

## 4. Parameter Importance (Random Forest)
Which parameters actually drive the performance?

In [ ]:
X = df[["pop_size", "crossover_prob", "mutation_prob", "selection_pressure"]]
y = df["hypervolume"]

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x=importances.values, y=importances.index, palette="viridis")
plt.title("Parameter Importance (Random Forest)")
plt.xlabel("Importance Score")
plt.show()

## 5. Partial Dependence Plots
Feature importance tells us *what* matters, but Partial Dependence Plots (PDP) tell us *how* it matters (the functional shape).

In [ ]:
features = ["pop_size", "crossover_prob", "mutation_prob", "selection_pressure"]

print("Computing Partial Dependence Plots...")
fig, ax = plt.subplots(figsize=(14, 4))
# Note: grid_resolution controls the smoothness
PartialDependenceDisplay.from_estimator(rf, X, features, ax=ax, grid_resolution=20)
plt.suptitle("Partial Dependence of Hypervolume on Parameters", fontsize=14, y=1.05)
plt.subplots_adjust(top=0.9)
plt.show()